# Metaprogramming in Julia

We'll write a program that writes programs. 

More precisely: it will translate DAGs into Julia functions.

In [1]:
# This is a simple directed graph module I wrote.
include("DiGraph.jl")
using .DiGraphs

In [2]:
# A simple type for defining operation nodes.
struct Node
    varname::Symbol
    op::Symbol
end

## XOR function as a DAG

![xor circuit diagram](xor-circuit-edited.png)

In [3]:
xor_edges = [[Node(:x1, :input) Node(:z1, :!)];
    [Node(:x2, :input) Node(:z2, :!)];
    [Node(:x1, :input) Node(:z4, :&)];
    [Node(:x2, :input) Node(:z3, :&)];
    [Node(:z1, :!) Node(:z3, :&)];
    [Node(:z2, :!) Node(:z3, :&)]
    [Node(:z3, :&) Node(:z5, :|)];
    [Node(:z4, :&) Node(:z5, :|)];
    [Node(:z5, :|) Node(:z6, :output)]
    ];

In [4]:
xor_dag = DiGraph(xor_edges);

## Translating from DAGs to Functions

### Vertices to operations

In [5]:
function make_expr(head, args)
    result = Expr(head)
    result.args = args
    return result
end


function opvertex_to_expr(v, dg)
    rhs = make_expr(:call, vcat([v.op], [p.varname for p in in_neighbors(dg, v)]))
    return make_expr(:(=), vcat([v.varname], rhs))
end


function retvertex_to_expr(v, dg)
    return make_expr(:return, [p.varname for p in in_neighbors(dg, v)])
end


function invertices_to_sig(invertices, dg, funcname::String)
    return make_expr(:call, vcat(Symbol(funcname), [v.varname for v in invertices]))
end


invertices_to_sig (generic function with 1 method)

### DAG to function

In [6]:
function dag_to_func(dg, funcname::String="myfunction")

    invertices = [v for v in dg.vertices if v.op == :input]
    
    # There must be at least one return vertex:
    retvertex = [v for v in dg.vertices if v.op == :output][1]

    opvertices = [v for v in topological_sort(dg) if (! in(v, invertices)) & (! (v == retvertex))]
    
    signature = invertices_to_sig(invertices, dg, funcname)
    operations = [opvertex_to_expr(v, dg) for v in opvertices]
    retline = [retvertex_to_expr(retvertex, dg)]
    
    body = make_expr(:block, vcat(operations, retline))
    return make_expr(:function, vcat([signature], [body]))
                                        
end

dag_to_func (generic function with 2 methods)

### Testing it on the XOR circuit

In [7]:
func_expr = dag_to_func(xor_dag, "xor_function")

:(function xor_function(x2, x1)
      z4 = (&)(x1)
      z2 = !x2
      z1 = !x1
      z3 = &(z1, x2, z2)
      z5 = z3 | z4
      return z5
  end)

In [8]:
eval(func_expr)

xor_function (generic function with 1 method)

In [9]:
for i = (true, false)
    for j = (true, false)
        println(i, "\t", j, "\t", xor_function(i,j))
    end
end

true	true	true
true	false	false
false	true	true
false	false	false
